In [ ]:
#初期設定
import os
import pandas as pd
import numpy as np
import time
import lxml
import re
import requests
from bs4 import BeautifulSoup as bs4
import tqdm
import sys
import inspect
# chromeのwebdriver自動更新用
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [ ]:
pd.set_option('display.max_rows', 30)
# pd.options.display.max_rows = 30
pd.set_option('display.max_columns', 20)

In [ ]:
# https://web-kiwami.com/python-beautyfulsoup4.html
# http://kondou.com/BS4/
# bs4参考

In [ ]:
# 1
# all_year
this_year = 2022 #年変わったら変更
get_year = list(range(2011,this_year+1))
df_scraping_companies = pd.DataFrame()
dict1 = {}
for year in get_year:
    if 2011 <= year <= 2017:
        print(year, "出力中")
        # 単年のデータ取得2011~2017
        url =  rf"https://www.ipokiso.com/company/{year}.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(len(dfs_list)):
            dfs_list[i]["上場年"] = f"{year}"
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list[i]])
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    else:
        # 単年のデータ取得2018~2022
        print(year, "出力中")
        url = rf"https://www.ipokiso.com/company/{year}.html"
        if year == 2022:
            url = r"https://www.ipokiso.com/company/index.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        # サイト更新時にページのtableのデザインが変更していないかチェック
        if year >= 2022:
            try:
                if dfs_list[0].columns[0] != '企業名':
                    raise ValueError("銘柄一覧から取得したtableの値が「企業名」ではありません")
                elif dfs_list[1].columns[0] != '総合評価':
                    raise ValueError("銘柄一覧から取得したtableの値が「総合評価」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")   
        time.sleep(np.random.randint(100,120)/100)
        for i in range(0, len(dfs_list) , 2):
            dfs_list_con = pd.concat([dfs_list[i], dfs_list[i+1]], axis=1)
            dfs_list_con["上場年"] = f"{year}"
            if year == 2019:
                dfs_list_con = dfs_list_con.rename(columns={'上場 市場': '上場市場'})
            if year >= 2020:
                dfs_list_con = dfs_list_con.rename(columns={'申し込み期間': '申し込み 期間', '初値上昇率': '初値 上昇率'})
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list_con],ignore_index=True)
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    if year != this_year:
        time.sleep(np.random.randint(100,120)/100)
#codeカラム作成
df_scraping_companies_code = df_scraping_companies["企業名"].str.extract(r'([0-9]{4})').fillna(0).astype(int)
df_scraping_companies["code"] = df_scraping_companies["銘柄 コード"].fillna(df_scraping_companies_code[0])
df_scraping_companies["code"] = df_scraping_companies["code"].astype(int)
# サイト側の入力間違えしているcodeを手動で訂正する 行指定でindex番号はずれる可能性があるため使わない
df_scraping_companies.loc[df_scraping_companies['企業名'] == "シグマクシス", 'code'] = 6088 #シグマクシス
df_scraping_companies.loc[df_scraping_companies['企業名'] == "アビスト 当選！", 'code'] = 6087 #アビスト
df_scraping_companies.loc[df_scraping_companies['企業名'] == "JESCO ホールディングス", 'code'] = 1434 #JESCO
df_scraping_companies.loc[df_scraping_companies['企業名'] == "じげん 当選！", 'code'] = 3679 #じげん

In [ ]:
# dict作成
# 空白削除
del dict1[" "]
del dict1[""]
del dict1["\n"]
# dictをdf化
df_scraping_url = pd.DataFrame.from_dict(dict1, orient='index')
# カラム名変更
df_scraping_url = df_scraping_url.rename(columns={0:"url"})
# 間違えているurlを変更
df_scraping_url.url[df_scraping_url.url == "https://www.ipokiso.com/https://www.ipokiso.com/company/2013/zigexn.html"] = "https://www.ipokiso.com/company/2013/zigexn.html"
df_scraping_url = df_scraping_url.rename_axis('index').reset_index()
df_scraping_url = df_scraping_url.rename(columns={'index':'company_name'})

In [ ]:
# 2
# seleniumでのデータ取得用ループ　評価
# chromeのwebdriverのinstallしブラウザを起動する 
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
def make_company_info(url):
    val_list = []
    driver.get(url)
    #seleniumの場合ここでsleepしないと正しく取得出来ない場合がある
    time.sleep(np.random.randint(100,120)/100)
    # HTMLを文字コードをUTF-8に変換してから取得します。
    html_res = driver.page_source.encode('utf-8')
#ここでシステムエラー処理する？
    # if html_res.status_code != 200:
    #     print("requests.getでのurlのアクセスができていません")
    #     lineno = inspect.currentframe().f_lineno
    #     print(f"エラーが発生しました。行番号: {lineno}")
    #     sys.exit()
    soup = bs4(html_res, 'html.parser')
    # 全企業codeの取得
    try:
        code = int(re.search(r"[0-9]{4}", soup.title.text).group())
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = int(re.search(r"[0-9]{4}", f).group())
    # 3171のtitleのcodeが3172と誤字のための修正
    if code == 3172:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = int(re.search(r"[0-9]{4}", f).group())
    val_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list.extend([np.nan,np.nan,np.nan,np.nan])
        print(f"成長性listのエラー{code}")
    print(code)
    # アンケート調査結果の取得 チェック用
    # print(len(soup.find_all("div",class_="vote-bar")))
    # 6行の場合
    if len(soup.find_all("div",class_="vote-bar")) == 6:
        vote_list = []
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 5行の場合
    elif len(soup.find_all("div",class_="vote-bar")) == 5:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 10行の場合 5行のアンケートがなぜか２つ表示されている銘柄
    elif len(soup.find_all("div",class_="vote-bar")) == 10:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        number_of_votes_list = number_of_votes_list[0:5]
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # アンケートがない場合
    elif len(soup.find_all("div",class_="vote-bar")) == 0:
        val_list.extend([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    # その他の行数エラー検知
    else:
        print("票の価格帯の数が分岐にありません")
        driver.quit()
        raise Exception("Error: アンケートの価格帯の数がifの分岐にありません")
    # 今まで取得した情報をリストに格納
    evaluation_list.append(val_list)
df_scraping_evaluation = df_scraping_url.copy()
url_list = df_scraping_url.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:]] #ここで検証のurl数変更！
print(evaluation_list)
df_company_01 = pd.DataFrame(evaluation_list,columns=[
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満",
])
df_scraping_evaluation[df_company_01.columns] = df_company_01
print(df_scraping_evaluation)
driver.quit()

In [ ]:
# 3
def make_company_info_2(url):
    dfs_table_list = pd.read_html(url)
    print(dfs_table_list[1].iloc[0,1])
    time.sleep(np.random.randint(100,120)/100)
    # サイト更新時にデザイン変更がないか検知する（現状ワシントンホテルのみ違うためスルー）
    # 回避urlリスト
    avoidance_url_list = [
        "https://www.ipokiso.com//company/2019/washingtonhotel.html",#[0]に１つ余分に優待情報のtableある
        "https://www.ipokiso.com//company/2020/gmo-fg.html",#table[6][7]が[7][8]にずれている
        "https://www.ipokiso.com//company/2021/frontier.html",#table[6]が無い
        "https://www.ipokiso.com//company/2021/geolocation.html",#table[6]が無い
        "https://www.ipokiso.com//company/2015/gmo-media.html",#table[6][7]が[7][8]にずれている
        ]
    if url not in avoidance_url_list:
        try:
            if dfs_table_list[1].iloc[0,0] != "会社名":
                raise ValueError("銘柄一覧から取得したtableの値が「会社名」ではありません")
            elif dfs_table_list[2].iloc[0,0] != '想定価格':
                raise ValueError("銘柄一覧から取得したtableの値が「想定価格」ではありません")               
            elif dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                raise ValueError("銘柄一覧から取得したtableの値が「抽選申込期間」ではありません")
            elif not '公募' in dfs_table_list[4].iloc[0,0]:
                raise ValueError("銘柄一覧から取得したtableの値が「公募株数等」ではありません")
            elif dfs_table_list[5].iloc[0,1] != '証券会社名':
                raise ValueError("銘柄一覧から取得したtableの値が「証券会社名」ではありません")
        except ValueError as e:
            # エラーが発生した場合の処理
            print("Error: {}".format(e))
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生した行番号: {lineno}")
            print(f"エラーが発生したURL:{url}")
            print(dfs_table_list)
            sys.exit()
        # 会社名にREITと投資法人という文字列が無いことを判定する
        if not ("REIT" in dfs_table_list[1].iloc[0,1]) and not ("投資法人" in dfs_table_list[1].iloc[0,1]):
            try:
                if dfs_table_list[6].columns[0] != '株主名':
                    raise ValueError("銘柄一覧から取得したtableの値が「株主名」ではありません")
                elif not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    raise ValueError("銘柄一覧から取得したtableの値が「財務データ等」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")
                print(f"エラーが発生したURL:{url}")
                print(dfs_table_list)
                sys.exit()

    # ここからdfsの結合、整形コード
    df_con_list = []
    for i in range(len(dfs_table_list)):
        # 基本情報
        if i == 1:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if df_table.iloc[0,0] != "会社名":
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：基本情報
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_1)
            # 表記の揺れ統一のための修正

        # IPO日程と価格決定（初値予想）
        if i == 2:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if dfs_table_list[2].iloc[0,0] != '想定価格':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：IPO日程と価格決定（初値予想）
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_2)
            # 表記の揺れ統一のための修正
            
        # IPOスケジュール
        if i == 3:
            # tableの位置指定 # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                df_table = dfs_table_list[i + 1]
                print(url)
            
            # df整形：IPOスケジュール
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_3 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_3)
            # 表記の揺れ統一のための修正
        # IPO当選株数
        if i == 4:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if not '公募' in dfs_table_list[4].iloc[0,0]:
                df_table = dfs_table_list[i + 1]
                print(url)
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = dfs_table_list[i + 1]
            
            # df整形：IPO当選株数
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_4 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_4)
            # 表記の揺れ統一のための修正

        # 幹事証券リスト（管理人独自予想あり）
        if i == 5:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[5].iloc[0,1] != '証券会社名':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：幹事証券リスト（管理人独自予想あり）
            df_table_5 = pd.DataFrame()
            for j in range(1,len(df_table.iloc[:,0])):
                val = df_table.iloc[j,:].values
                cols = df_table.iloc[0,:]
                cols[0] = "幹事種類"
                cols = [f"{col}_{j}" for col in cols]
                df_add = pd.DataFrame([val],columns=cols)
                df_table_5 = pd.concat([df_table_5, df_add], axis=1)
            df_con_list.append(df_table_5)

        # 株主構成、ロックアップなど
        if i == 6:
            if dfs_table_list[1].iloc[0,0] == "会社名":
                company_name = dfs_table_list[1].iloc[0,1]
            elif dfs_table_list[2].iloc[0,0] == "会社名":
                company_name = dfs_table_list[2].iloc[0,1]
            if not ("REIT" in company_name or "投資法人" in company_name or  ("福証" in company_name and not "東証" in company_name)):
                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if dfs_table_list[6].columns[0] != '株主名':
                    df_table = dfs_table_list[i + 1]
                    print(url)
                df_table_6 = pd.DataFrame()
                for k in range(0,len(df_table.iloc[:,0])):
                    val = df_table.iloc[k,:].values
                    cols = df_table.columns
                    cols = [f"{col}_{k+1}" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_6 = pd.concat([df_table_6, df], axis=1)
                df_con_list.append(df_table_6)

            # 企業業績のデータ（5年分）
        if i == 7:
            if not ("REIT" in company_name or "投資法人" in company_name):

                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    df_table = dfs_table_list[i + 1]
                    print(url)
                
                df_table_7 = pd.DataFrame()
                for h in range(1,len(df_table.T.iloc[:,0])):
                    val = df_table.T.iloc[h,:].values
                    # カラム名の誤字修正
                    cols = df_table.T.iloc[0,:]
                    cols[0] = "年月"
                    cols = [f"{col}_{h}年目" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_7 = pd.concat([df_table_7, df], axis=1)
                df_con_list.append(df_table_7)


            # 全件のdfに追加していく
    df_table_all = pd.concat(df_con_list,axis=1)
    return df_table_all
df_scraping_table_all = pd.DataFrame()
url_list = df_scraping_url.url.values
for url in url_list[0:]:#ここでurl数変更
    df_table_all = make_company_info_2(url) 
    df_scraping_table_all = pd.concat([df_scraping_table_all,df_table_all],axis=0,ignore_index=True)

In [ ]:
# 時短のためcode含めたcsv読み込み　コード完成後に削除する
# df_scraping_evaluation = pd.read_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_url.csv", index_col=0)
# df_scraping_evaluation

全スクレイピングのdf作成　df_scraping_all


In [ ]:
#  cocat df_scraping_eva_tab作成
df_scraping_eva_tab = pd.concat([df_scraping_evaluation,df_scraping_table_all],axis=1)

In [ ]:
# merge df_scraping_all作成 これがスクレイピング全データでこれを元にoutputのdf作る。
df_scraping_all = pd.merge(df_scraping_eva_tab, df_scraping_companies, on="code", how="inner", suffixes=("","_comp"))
df_scraping_all = df_scraping_all.drop_duplicates()

In [ ]:
# 上場中止銘柄とサイト側のミスデータの行削除 文字列に中止等があれば削除
df_scraping_all = df_scraping_all[~df_scraping_all['企業名'].str.contains('中止')]
df_scraping_all = df_scraping_all[~df_scraping_all['上場日_comp'].str.contains('中止')]
df_scraping_all = df_scraping_all[~df_scraping_all['company_name'].str.contains('ヤマイチエステート')]#ミスデータ
df_scraping_all = df_scraping_all[~df_scraping_all['company_name'].str.contains('テノ．\r\nホールディングス')]#ミスデータ
df_scraping_all = df_scraping_all[~df_scraping_all['company_name'].str.contains('じげん（3679）\n')]#ミスデータ
df_scraping_all = df_scraping_all.reset_index(drop=True)

In [ ]:
# チェック用
df_scraping_all[df_scraping_all["code"]==3679]

In [ ]:
df_scraping_all

次はここから！2022/12/21予定
1,文字整形
2,分割

最終出力dfの4つ作成 
df_assessments
df_schedule
df_underwriters
df_shareholders

In [ ]:
# df_scraping_all.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_all.csv", encoding='utf-8_sig',index=False)

In [ ]:
# 評価+ 会社設立年 + 市場(G,P,S,…)　+ IPO日程と価格決定(初値予測) + 初値予想アンケート output_data1/4
df_assessments = df_scraping_all[[
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満",
    "会社設立",
    "会社名",#市場に変形する
    "上場市場",
    "想定価格",
    "仮条件",
    "公募価格",
    "初値予想（独自）",
    "初値"
]]
# 会社設立の値を西暦4桁のみに置換(float型)
df_assessments["会社設立"] = df_assessments["会社設立"].str.extract(r'([0-9]{4})').fillna(np.nan).astype(float)
# 分割　（二つに列を分ける場合）
df_assessments = df_assessments.assign(A=df_assessments['仮条件'].str.split('～|-').str[0])
df_assessments = df_assessments.assign(B=df_assessments['仮条件'].str.split('～|-').str[1])
df_assessments = df_assessments.assign(C=df_assessments['初値予想（独自）'].str.split('～').str[0])
df_assessments = df_assessments.assign(D=df_assessments['初値予想（独自）'].str.split('～').str[1])
# 分割　（置換ver）　初値（円）分割して代入
df_assessments["初値"] = df_assessments["初値"].str.split('円').str[0]
# rename
df_assessments = df_assessments.rename(columns={
    "想定価格":"想定価格（円）",
    "仮条件":"仮条件（円）",
    "A":"仮条件（円）_下値",
    "B":"仮条件（円）_上値",
    "公募価格":"公募価格（円）",
    "C":"初値予想（独自）（円）_下値",
    "D":"初値予想（独自）（円）_上値",
    "初値":"初値（円）",
    })
# . ,  円の削除
df_assessments['想定価格（円）'] = df_assessments['想定価格（円）'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['公募価格（円）'] = df_assessments['公募価格（円）'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['初値（円）'] = df_assessments['初値（円）'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['仮条件（円）_下値'] = df_assessments['仮条件（円）_下値'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['仮条件（円）_上値'] = df_assessments['仮条件（円）_上値'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['初値予想（独自）（円）_下値'] = df_assessments['初値予想（独自）（円）_下値'].str.replace('～','').str.replace(',','').str.replace('.','')
df_assessments['初値予想（独自）（円）_上値'] = df_assessments['初値予想（独自）（円）_上値'].str.replace('～','').str.replace(',','').str.replace('.','')
# 列にある文字列を削除して、float型に変える（ついでに空欄がなくなる）
df_assessments['想定価格（円）'] = df_assessments['想定価格（円）'].str.extract(r'(\d+)').astype(float)
df_assessments['公募価格（円）'] = df_assessments['公募価格（円）'].str.extract(r'(\d+)').astype(float)
df_assessments['初値（円）'] = df_assessments['初値（円）'].str.extract(r'(\d+)').astype(float)
df_assessments['仮条件（円）_下値'] = df_assessments['仮条件（円）_下値'].str.extract(r'(\d+)').astype(float)
df_assessments['仮条件（円）_上値'] = df_assessments['仮条件（円）_上値'].str.extract(r'(\d+)').astype(float)
df_assessments['初値予想（独自）（円）_下値'] = df_assessments['初値予想（独自）（円）_下値'].str.extract(r'(\d+)').astype(float)
df_assessments['初値予想（独自）（円）_上値'] = df_assessments['初値予想（独自）（円）_上値'].str.extract(r'(\d+)').astype(float)
# 欠損値をうめる　予想範囲がなく下値のみのものを上値にも同額いれる
df_assessments['仮条件（円）_上値'] = df_assessments['仮条件（円）_上値'].fillna(df_assessments['仮条件（円）_下値'])
df_assessments['初値予想（独自）（円）_上値'] = df_assessments['初値予想（独自）（円）_上値'].fillna(df_assessments['初値予想（独自）（円）_下値'])
# 不必要な列drop
df_assessments = df_assessments.drop("仮条件（円）", axis=1)
df_assessments = df_assessments.drop("初値予想（独自）", axis=1)

In [ ]:
# IPOスケジュール + IPO当選株数 output2/4
df_schedule = df_scraping_all[[
    "code",
    "上場年",
    "抽選申込期間",
    "当選発表日",
    "購入申込期間",
    "上場日",
    "公募株数",
    "売出株数（OA含む）",
    "当選株数合計"
]]
# 分割　（二つに列を分ける場合）
df_schedule = df_schedule.assign(A=df_schedule["抽選申込期間"].str.split('～|-').str[0])
df_schedule = df_schedule.assign(B=df_schedule["抽選申込期間"].str.split('～|-').str[1])
df_schedule = df_schedule.assign(C=df_schedule["購入申込期間"].str.split('～|-').str[0])
df_schedule = df_schedule.assign(D=df_schedule["購入申込期間"].str.split('～|-').str[1])
df_schedule = df_schedule.assign(E=df_schedule["公募株数"].str.split('株|口').str[0])
df_schedule = df_schedule.assign(F=df_schedule["売出株数（OA含む）"].str.split('株|口').str[0])
df_schedule = df_schedule.assign(G=df_schedule["当選株数合計"].str.split('株|口').str[0])
# rename
df_schedule = df_schedule.rename(columns={
    "A":"抽選申込期間_開始日",
    "B":"抽選申込期間_終了日",
    "C":"購入申込期間_開始日",
    "D":"購入申込期間_終了日",
    "E":"公募株数（株）",
    "F":"売出株数（OA含む）（株）",
    "G":"当選株数合計（株）",
    })
# . ,  円の削除
df_schedule['公募株数（株）'] = df_schedule['公募株数（株）'].str.replace('円','').str.replace(',','').str.replace('.','').astype(float)
df_schedule['売出株数（OA含む）（株）'] = df_schedule['売出株数（OA含む）（株）'].str.replace('円','').str.replace(',','').str.replace('.','').astype(float)
df_schedule['当選株数合計（株）'] = df_schedule['当選株数合計（株）'].str.replace('円','').str.replace(',','').str.replace('.','').astype(float)
# 日付をpd.to_datetimeで読み取れるように文字列の形を統一する
df_schedule["当選発表日"] = df_schedule['当選発表日'].apply(lambda x: re.search(r'(\d+月\d+日)', x).group(1) if re.search(r'(\d+月\d+日)', x) else "N/A")
df_schedule["上場日"] = df_schedule['上場日'].apply(lambda x: re.search(r'(\d+月\d+日)', x).group(1) if re.search(r'(\d+月\d+日)', x) else "N/A")
df_schedule["抽選申込期間_開始日"] = df_schedule['抽選申込期間_開始日'].apply(lambda x: re.search(r'(\d+月\d+日)', x).group(1) if re.search(r'(\d+月\d+日)', x) else "N/A")
df_schedule['抽選申込期間_終了日'] = df_schedule['抽選申込期間_終了日'].apply(lambda x: re.search(r'(\d+月\d+日)', x).group(1) if re.search(r'(\d+月\d+日)', x) else "N/A")
df_schedule["購入申込期間_開始日"] = df_schedule['購入申込期間_開始日'].apply(lambda x: re.search(r'(\d+月\d+日)', x).group(1) if re.search(r'(\d+月\d+日)', x) else "N/A")
df_schedule["購入申込期間_終了日"] = df_schedule['購入申込期間_終了日'].apply(lambda x: re.search(r'(\d+月\d+日)', x).group(1) if re.search(r'(\d+月\d+日)', x) else "N/A")
# 年と月日の文字列を合成
df_schedule["当選発表日"] = df_schedule.apply(lambda row: row['上場年'] + '年' + row['当選発表日'], axis=1)
df_schedule["上場日"] = df_schedule.apply(lambda row: row['上場年'] + '年' + row['上場日'], axis=1)
df_schedule["抽選申込期間_開始日"] = df_schedule.apply(lambda row: row['上場年'] + '年' + row['抽選申込期間_開始日'], axis=1)
df_schedule["抽選申込期間_終了日"] = df_schedule.apply(lambda row: row['上場年'] + '年' + row['抽選申込期間_終了日'], axis=1)
df_schedule["購入申込期間_開始日"] = df_schedule.apply(lambda row: row['上場年'] + '年' + row['購入申込期間_開始日'], axis=1)
df_schedule["購入申込期間_終了日"] = df_schedule.apply(lambda row: row['上場年'] + '年' + row['購入申込期間_終了日'], axis=1)
# 日付（年月日）をdatatime型に変換
df_schedule["当選発表日"] = pd.to_datetime(df_schedule["当選発表日"], format="%Y年%m月%d日", errors="coerce")
df_schedule["上場日"] = pd.to_datetime(df_schedule["上場日"], format="%Y年%m月%d日", errors="coerce")
df_schedule["抽選申込期間_開始日"] = pd.to_datetime(df_schedule["抽選申込期間_開始日"], format="%Y年%m月%d日", errors="coerce")
df_schedule["抽選申込期間_終了日"] = pd.to_datetime(df_schedule["抽選申込期間_終了日"], format="%Y年%m月%d日", errors="coerce")
df_schedule["購入申込期間_開始日"] = pd.to_datetime(df_schedule["購入申込期間_開始日"], format="%Y年%m月%d日", errors="coerce")
df_schedule["購入申込期間_終了日"] = pd.to_datetime(df_schedule["購入申込期間_終了日"], format="%Y年%m月%d日", errors="coerce")
# 不必要な列drop
df_schedule = df_schedule.drop("公募株数", axis=1)
df_schedule = df_schedule.drop("売出株数（OA含む）", axis=1)
df_schedule = df_schedule.drop("当選株数合計", axis=1)
df_schedule = df_schedule.drop("抽選申込期間", axis=1)
df_schedule = df_schedule.drop("購入申込期間", axis=1)
df_schedule = df_schedule.drop("上場年", axis=1)

途中まで実施　次回2022/12/26～
・文字が含まれる列の値を削除
・float型に変更する？列の型統一
・欠損値の値の統一

In [418]:
# 幹事証券リスト（管理人独自予想あり）output3/4
df_underwriters = df_scraping_all.filter(regex="code|幹事種類_|証券会社名_|割当率_|割当株数_|当選本数 （枚）_|完全抽選本数 （予想）_")
# カラムをcodeと1~10のもののみフィルターで残す
df_underwriters = df_underwriters.filter(regex="code|_1$|_2$|_3$|_4$|_5$|_6$|_7$|_8$|_9$|_10$")
# 特定の文字列が入ったカラム名をグループ化
column_names = df_underwriters.columns.tolist()
# グループ
columns_with_幹事種類 = [column_name for column_name in column_names if '幹事種類' in column_name]
columns_with_証券会社名 = [column_name for column_name in column_names if '証券会社名' in column_name]
columns_with_割当率 = [column_name for column_name in column_names if '割当率' in column_name]
columns_with_割当株数 = [column_name for column_name in column_names if '割当株数' in column_name]
columns_with_当選本数 = [column_name for column_name in column_names if '当選本数' in column_name]
columns_with_完全抽選本数 = [column_name for column_name in column_names if '完全抽選本数' in column_name]
# 文字列操作のためにグループごとに全ての値を文字列に加工 値一つ一つに対してstr()関数を実行している　欠損値があるとnan(float)になってしまうため必要な処理
df_underwriters[columns_with_割当率] = df_underwriters[columns_with_割当率].applymap(str)
df_underwriters[columns_with_割当株数] = df_underwriters[columns_with_割当株数].applymap(str)
df_underwriters[columns_with_当選本数] = df_underwriters[columns_with_当選本数].applymap(str)
df_underwriters[columns_with_完全抽選本数] = df_underwriters[columns_with_完全抽選本数].applymap(str)
# グループごとに全ての文字列を加工　関数が無ければ無名関数でつくる
df_underwriters[columns_with_割当率] = df_underwriters[columns_with_割当率].applymap(lambda x: x.replace('％', '').replace('%', '').replace('-', ''))
df_underwriters[columns_with_割当株数] = df_underwriters[columns_with_割当株数].applymap(lambda x: x.replace('株', '').replace('口', '').replace(',', '').replace('.', '').replace('-', ''))
df_underwriters[columns_with_当選本数] = df_underwriters[columns_with_当選本数].applymap(lambda x: x.replace('枚', '').replace(',', '').replace('.', '').replace('-', ''))
df_underwriters[columns_with_完全抽選本数] = df_underwriters[columns_with_完全抽選本数].applymap(lambda x: x.replace('枚', '').replace(',', '').replace('.', '').replace('-', ''))
# もし文字があった場合に.と数字以外を全て消す　数字ではないコメントがあるとfloat型に変換できないため
df_underwriters[columns_with_割当率] = df_underwriters[columns_with_割当率].applymap(lambda x: re.sub(r'[^\d\.]', '', x))
df_underwriters[columns_with_割当株数] = df_underwriters[columns_with_割当株数].applymap(lambda x: re.sub(r'[^\d\.]', '', x))
df_underwriters[columns_with_当選本数] = df_underwriters[columns_with_当選本数].applymap(lambda x: re.sub(r'[^\d\.]', '', x))
df_underwriters[columns_with_完全抽選本数] = df_underwriters[columns_with_完全抽選本数].applymap(lambda x: re.sub(r'[^\d\.]', '', x))
# 列の型をfloat型に変換 columns_with_割当率は比率のため×0.01
df_underwriters[columns_with_割当率] = df_underwriters[columns_with_割当率].applymap(lambda x: float(x) * 0.01 if x != '' else np.nan)
df_underwriters[columns_with_割当株数] = df_underwriters[columns_with_割当株数].applymap(lambda x: float(x) if x != '' else np.nan)
df_underwriters[columns_with_当選本数] = df_underwriters[columns_with_当選本数].applymap(lambda x: float(x) if x != '' else np.nan)
df_underwriters[columns_with_完全抽選本数] = df_underwriters[columns_with_完全抽選本数].applymap(lambda x: float(x) if x != '' else np.nan)

,code,幹事種類_1,証券会社名_1,割当率_1,割当株数_1,当選本数 （枚）_1,完全抽選本数 （予想）_1,幹事種類_2,証券会社名_2,割当率_2,...,割当率_9,割当株数_9,当選本数 （枚）_9,完全抽選本数 （予想）_9,幹事種類_10,証券会社名_10,割当率_10,割当株数_10,当選本数 （枚）_10,完全抽選本数 （予想）_10
0,3170,主幹事,大和証券,0.7522,187300.0,1873.0,187.0,幹事,SMBC日興証券,0.1498,...,NaN,NaN,NaN,NaN,幹事,NaN,NaN,NaN,NaN,NaN
1,3169,主幹事,みずほ インベスターズ 証券,0.8000,480000.0,4800.0,480.0,幹事,SMBC日興証券,0.1000,...,NaN,NaN,NaN,NaN,幹事,NaN,NaN,NaN,NaN,NaN
2,9206,主幹事,大和証券,0.7500,150000.0,1500.0,225.0,幹事,SMBC日興証券,0.0500,...,0.0100,2000.0,20.0,2.0,幹事,エイチエス証券,0.0100,2000.0,20.0,10.0
3,4977,主幹事,SMBC日興証券,0.7499,2720700.0,27207.0,2720.0,幹事,野村證券,0.0900,...,0.0100,36300.0,363.0,36.0,幹事,NaN,NaN,NaN,NaN,NaN
4,4583,主幹事,大和証券,0.8004,621900.0,6219.0,932.0,幹事,みずほ証券,0.0400,...,0.0099,7700.0,77.0,7.0,幹事,エース証券,0.0099,7700.0,77.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,9215,主幹事,SBI証券,0.8693,211500.0,2115.0,951.0,幹事,みずほ証券,0.0781,...,0.0045,1100.0,11.0,0.0,幹事,水戸証券,0.0045,1100.0,11.0,0.0
987,4268,主幹事,SMBC日興証券,0.9305,2899900.0,28999.0,2899.0,幹事,野村證券,0.0435,...,NaN,NaN,NaN,NaN,幹事,NaN,NaN,NaN,NaN,NaN
988,4267,主幹事,エイチ・エス証券,0.8435,310400.0,3104.0,310.0,幹事,SMBC日興証券,0.0522,...,NaN,NaN,NaN,NaN,幹事,NaN,NaN,NaN,NaN,NaN
989,9213,主幹事,みずほ証券,0.9567,820300.0,8203.0,820.0,幹事,大和証券,0.0174,...,NaN,NaN,NaN,NaN,幹事,NaN,NaN,NaN,NaN,NaN


In [516]:
# 株主構成 + ロックアップ[6]　output4/4
df_shareholders = df_scraping_all.filter(regex="code|株主名_|比率_|ロック アップ_|ロック  アップ_")
# カラムをcodeと1~10のもののみフィルターで残す
df_shareholders = df_shareholders.filter(regex="code|_1$|_2$|_3$|_4$|_5$|_6$|_7$|_8$|_9$|_10$")
# 特定の文字列が入ったカラム名をグループ化
column_names = df_shareholders.columns.tolist()
# グループ
columns_with_株主名 = [column_name for column_name in column_names if '株主名' in column_name]
columns_with_比率 = [column_name for column_name in column_names if '比率' in column_name]
columns_with_ロックアップ1 = [column_name for column_name in column_names if 'ロック  アップ' in column_name]
columns_with_ロックアップ2 = [column_name for column_name in column_names if 'ロック アップ' in column_name]
# 2列に分かれているロックアップの合成
# 複数の列のfillna、zipにより複数のリスト同士を反復させる。
for a, b in zip(columns_with_ロックアップ1, columns_with_ロックアップ2):
    df_shareholders[a] = df_shareholders[a].fillna(df_shareholders[b])
# 文字列操作のためにグループごとに全ての値を文字列に加工 値一つ一つに対してstr()関数を実行している　欠損値があるとnan(float)になってしまうため必要な処理
# 文字列のみの列はやらない
df_shareholders[columns_with_比率] = df_shareholders[columns_with_比率].applymap(str)
df_shareholders[columns_with_ロックアップ1] = df_shareholders[columns_with_ロックアップ1].applymap(str)
# 分割　ロックアップ解除の列二つ作成　日の前の数値、倍の前の値をとっている　要文字列削除
for column in columns_with_ロックアップ1:
    df_shareholders[column + '_解除日数'] = df_shareholders[column].str.split('日').str[0]
    df_shareholders[column + '_解除倍率'] = df_shareholders[column].str.split('日').str[-1]
# グループ追加
column_names = df_shareholders.columns.tolist()
columns_with_解除日数 = [column_name for column_name in column_names if '_解除日数' in column_name]
columns_with_解除倍率 = [column_name for column_name in column_names if '_解除倍率' in column_name]
# 文字列操作
df_shareholders[columns_with_比率] = df_shareholders[columns_with_比率].applymap(lambda x: x.replace('％', '').replace('%', '').replace('-', ''))

# もし文字があった場合に.と数字以外を全て消す　数字ではないコメントがあるとfloat型に変換できないため
df_shareholders[columns_with_比率] = df_shareholders[columns_with_比率].applymap(lambda x: re.sub(r'[^\d\.]', '', x))
df_shareholders[columns_with_解除日数] = df_shareholders[columns_with_解除日数].applymap(lambda x: re.sub(r'[^\d\.]', '', x))
df_shareholders[columns_with_解除倍率] = df_shareholders[columns_with_解除倍率].applymap(lambda x: re.sub(r'[^\d\.]', '', x))
# 列の型をfloat型に変換 columns_with_割当率は比率のため×0.01
df_shareholders[columns_with_比率] = df_shareholders[columns_with_比率].applymap(lambda x: float(x) * 0.01 if x != '' else np.nan)
df_shareholders[columns_with_解除日数] = df_shareholders[columns_with_解除日数].applymap(lambda x: float(x) if x != '' else np.nan)
df_shareholders[columns_with_解除倍率] = df_shareholders[columns_with_解除倍率].applymap(lambda x: float(x) if x != '' else np.nan)
# 不必要な列drop
df_shareholders = df_shareholders.drop(labels=columns_with_ロックアップ1, axis=1)
df_shareholders = df_shareholders.drop(labels=columns_with_ロックアップ2, axis=1)

,code,株主名_1,比率_1,株主名_2,比率_2,株主名_3,比率_3,株主名_4,比率_4,株主名_5,...,ロック アップ_6_解除日数,ロック アップ_6_解除倍率,ロック アップ_7_解除日数,ロック アップ_7_解除倍率,ロック アップ_8_解除日数,ロック アップ_8_解除倍率,ロック アップ_9_解除日数,ロック アップ_9_解除倍率,ロック アップ_10_解除日数,ロック アップ_10_解除倍率
0,3170,岡村 幸彦（社長）,0.3041,（株）望未,0.1827,メディパルホールディングス,0.1827,中央エム・リース,0.0914,SMBCキャピタル８号,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3169,三沢 太（社長）,0.7615,飯塚 智香,0.0796,尾張 睦,0.0636,大塚 幸江,0.0409,従業員持ち株会,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9206,"DCM IV, L.P.",0.1726,ＴＯＴＯ,0.0508,安川電機,0.0329,北九州エアターミナル,0.0290,九州電力,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4977,アイビーピー,0.2510,ニッタ,0.0660,三菱東京ＵＦＪ銀行,0.0488,三井住友銀行,0.0471,（有）以和貴,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4583,ファストトラックイニシアティブ１号,0.1889,藤原 正明（社長）,0.1017,オリンパスビジネスクリエイツ,0.0859,太田 邦史,0.0565,清田 圭一,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,9215,加茂 雄一（社長）,0.2156,ワタキューセイモア（株）,0.1911,池田 裕樹,0.1775,胡桃 沢精一,0.0796,（株）I.K.D,...,90.0,1.5,90.0,1.5,90.0,1.5,90.0,1.5,90.0,1.5
987,4268,住本 幸士（社長）,0.9356,島田 雄太,0.0309,治田 知明,0.0120,坂西 茂,0.0039,生澤 剛士,...,180.0,NaN,180.0,NaN,180.0,NaN,180.0,NaN,180.0,NaN
988,4267,（株）エプシモーヴェ,0.5321,江口 夏郎（社長）,0.1587,小迫 宏行,0.0733,ライトワークス従業員持株会,0.0462,松林 洋太,...,90.0,1.5,180.0,NaN,90.0,1.5,NaN,NaN,NaN,NaN
989,9213,（株）ビューティープロスペリティー,0.3228,長谷川 高志（社長）,0.2324,長谷川 美栄,0.0959,山田 実,0.0691,藤本 宏志,...,90.0,1.5,180.0,NaN,180.0,NaN,180.0,NaN,180.0,NaN


以上本番用

In [517]:
df_shareholders.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_shareholders3.csv", encoding='utf-8_sig',index=False)

In [ ]:
# チェック用
df_scraping_all[df_scraping_all['code']==4932]

In [ ]:
# 重複チェック
# df_scraping_all['code'].value_counts(ascending=False)[0:10]

In [ ]:
# 時短のためcode含めたcsv読み込み
# df_scraping_evaluation = pd.read_csv("df_scraping_url.csv", index_col=0)
# df_scraping_evaluation